In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install transformers datasets peft wandb sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from peft import LoraConfig, get_peft_model
import pandas as pd
from sklearn.model_selection import train_test_split
import torch

In [6]:
# Check GPU availability
if torch.cuda.is_available():
    print("GPU is available!")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available. Please enable it in runtime settings.")

GPU is available!
GPU name: NVIDIA A100-SXM4-40GB


In [7]:
import torch
torch.cuda.empty_cache()

# Load Dataset

In [8]:
# File paths
training_set = "/content/drive/MyDrive/P2/T1/Dataset/combined/LLM-c-train.csv"
validation_set = "/content/drive/MyDrive/P2/T1/Dataset/combined/LLM-c-val.csv"
testing_set = "/content/drive/MyDrive/P2/T1/Dataset/combined/LLM-c-test.csv"

# Load datasets
train_data = pd.read_csv(training_set)
val_data = pd.read_csv(validation_set)
test_data = pd.read_csv(testing_set)

# Add prompt column to the datasets
for dataset in [train_data, val_data, test_data]:
    dataset['prompt'] = dataset.apply(
        lambda row: f"Symptoms: {row['symptoms']}\nPredict the disease:",
        axis=1
    )

# Create mappings for text-based labels
unique_labels = train_data['disease_label'].unique()
label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
id_to_label = {idx: label for idx, label in enumerate(unique_labels)}

# Map text labels to numeric indices
for dataset in [train_data, val_data, test_data]:
    dataset['target'] = dataset['disease_label'].map(label_to_id)

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")

disease_counts = train_data['disease_label'].value_counts()
print("Disease Distribution in Training Data:")
print(disease_counts)


Number of training samples: 50944
Number of validation samples: 9646
Number of testing samples: 9718
Disease Distribution in Training Data:
disease_label
hypertensive disease                  1530
biliary calculus                      1491
hyperlipidemia                        1464
coronary arteriosclerosis             1385
colitis                               1303
paroxysmal dyspnea                    1270
failure heart                         1120
osteomyelitis                         1115
infection                             1093
pyelonephritis                        1068
malignant neoplasms                   1033
gastritis                             1007
stenosis aortic valve                  994
overload fluid                         978
delirium                               964
hepatitis                              962
pancreatitis                           958
deep vein thrombosis                   943
manic disorder                         936
dependence                   

# Load LLM

In [9]:
# Import the required classes
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load ClinicalBERT model and tokenizer
model_name = "emilyalsentzer/Bio_ClinicalBERT"  # ClinicalBERT model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(unique_labels)
)

print("ClinicalBERT model and tokenizer loaded successfully!")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ClinicalBERT model and tokenizer loaded successfully!


# LoRA (Low-Rank Adaptation)

In [10]:

# Configure LoRA for BioBERT
lora_config = LoraConfig(
    r=8,  # Low-rank adaptation rank
    lora_alpha=32,  # Scaling factor
    target_modules=["query", "value"],  # LoRA applied to attention layers
    lora_dropout=0.1,  # Dropout rate for regularization
    task_type="SEQ_CLS"  # Sequence classification task
)

model = get_peft_model(model, lora_config)
print("LoRA applied successfully to ClinicalBERT!")


LoRA applied successfully to ClinicalBERT!


# Tokenization function

In [11]:
# Tokenization function for the dataset
def tokenize_function(examples):
    # Tokenize the 'prompt' column
    tokenized_inputs = tokenizer(
        examples['prompt'],  # Input text (symptoms prompt)
        truncation=True,
        padding='max_length',
        max_length=256,  # Adjust as needed
        return_tensors="pt"
    )
    # Add 'labels' field (numeric targets)
    tokenized_inputs["labels"] = examples["target"]
    return tokenized_inputs

# Apply tokenization to the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set the dataset format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


Map:   0%|          | 0/50944 [00:00<?, ? examples/s]

Map:   0%|          | 0/9646 [00:00<?, ? examples/s]

Map:   0%|          | 0/9718 [00:00<?, ? examples/s]

# Training Arguments

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results_clinicalbert',
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    num_train_epochs= 6,
    weight_decay=0.001,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir='./logs_clinicalbert',
    logging_steps=10,
    fp16=True,  # Mixed precision
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

<ipython-input-13-bc59b6678d02>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.798700,1.635716
2,1.048800,0.865345
3,0.899500,0.579327
4,0.614700,0.471171
5,0.653300,0.423315
6,0.597000,0.401988


TrainOutput(global_step=19104, training_loss=1.1884358491330491, metrics={'train_runtime': 1124.9354, 'train_samples_per_second': 271.717, 'train_steps_per_second': 16.982, 'total_flos': 4.039213123790438e+16, 'train_loss': 1.1884358491330491, 'epoch': 6.0})

# Training and Validation Loss Graph

In [15]:
"""
import matplotlib.pyplot as plt

# Data from the table
epochs = [1, 2]
training_loss = [2.1982, 2.4477, 1.8128, 1.3208, 1.1841, 1.1038]
validation_loss = [1.836972, 2.2917, 1.5439, 1.1537, 0.9580, 0.9067]

# Plot the graph
plt.figure(figsize=(10, 6))
plt.plot(epochs, training_loss, label="Training Loss", marker="o")
plt.plot(epochs, validation_loss, label="Validation Loss", marker="o")

# Add labels, title, and legend
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss Over Epochs")
plt.legend()
plt.grid(True)

# Show the plot
plt.show()
"""

'\nimport matplotlib.pyplot as plt\n\n# Data from the table\nepochs = [1, 2]\ntraining_loss = [2.1982, 2.4477, 1.8128, 1.3208, 1.1841, 1.1038]\nvalidation_loss = [1.836972, 2.2917, 1.5439, 1.1537, 0.9580, 0.9067]\n\n# Plot the graph\nplt.figure(figsize=(10, 6))\nplt.plot(epochs, training_loss, label="Training Loss", marker="o")\nplt.plot(epochs, validation_loss, label="Validation Loss", marker="o")\n\n# Add labels, title, and legend\nplt.xlabel("Epoch")\nplt.ylabel("Loss")\nplt.title("Training and Validation Loss Over Epochs")\nplt.legend()\nplt.grid(True)\n\n# Show the plot\nplt.show()\n'

In [16]:
"""
model.save_pretrained('/content/drive/MyDrive/P2/LLM/fine_tuned_biobert')
tokenizer.save_pretrained('/content/drive/MyDrive/P2/LLM/fine_tuned_biobert')
print("Fine-tuned BioBERT model and tokenizer saved successfully!")
"""

'\nmodel.save_pretrained(\'/content/drive/MyDrive/P2/LLM/fine_tuned_biobert\')\ntokenizer.save_pretrained(\'/content/drive/MyDrive/P2/LLM/fine_tuned_biobert\')\nprint("Fine-tuned BioBERT model and tokenizer saved successfully!")\n'

# Evaluate the fine-tuned model directly

In [17]:
from datasets import Dataset

# Load the test dataset
test_file_path = '/content/drive/MyDrive/P2/T1/Dataset/combined/LLM-c-test.csv'
test_data = pd.read_csv(test_file_path)

# Add prompt column (if not already present)
test_data['prompt'] = test_data.apply(
    lambda row: f"Symptoms: {row['symptoms']}\nPredict the disease:",
    axis=1
)

# Convert to Hugging Face Dataset
test_dataset = Dataset.from_pandas(test_data)

In [18]:
import re

def predict_disease(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=256).to("cuda")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_id = torch.argmax(logits, dim=-1).item()
        return id_to_label[predicted_class_id]  # Map numeric index back to text label

# Test Predictions
test_data['predicted_disease'] = test_data['prompt'].apply(predict_disease)

# Print Predictions
for index, row in test_data.iterrows():
    print(f"Prompt: {row['prompt']}")
    print(f"Predicted Disease: {row['predicted_disease']}")


Streaming output truncated to the last 5000 lines.
Predict the disease:
Predicted Disease: infection
Prompt: Symptoms: Patient shows symptoms as follows: have stiffness; have polydypsia; have numbness of hand.
Predict the disease:
Predicted Disease: degenerative polyarthritis
Prompt: Symptoms: Patient shows symptoms as follows: have hypothermia, natural; have unconscious state; have clammy skin; have claudication; have qt interval prolonged.
Predict the disease:
Predicted Disease: hypoglycemia
Prompt: Symptoms: Patient shows symptoms as follows: have palpitation; have nausea; have cushingoid habitus; have diarrhea.
Predict the disease:
Predicted Disease: biliary calculus
Prompt: Symptoms: Patient shows symptoms as follows: have pain chest; have nervousness; have apyrexial; have hemiplegia; have gurgle.
Predict the disease:
Predicted Disease: ulcer peptic
Prompt: Symptoms: Patient shows symptoms as follows: have yellow sputum; have inappropriate affect; have abdominal tenderness; have w

# Save fine-tuned Model

In [19]:
"""
# Save the fine-tuned model
model.save_pretrained('/content/drive/MyDrive/P2/LLM/fine_tuned_biogpt')
tokenizer.save_pretrained('/content/drive/MyDrive/P2/LLM/fine_tuned_biogpt')
print("Fine-tuned BioGPT model and tokenizer saved successfully!")
"""

'\n# Save the fine-tuned model\nmodel.save_pretrained(\'/content/drive/MyDrive/P2/LLM/fine_tuned_biogpt\')\ntokenizer.save_pretrained(\'/content/drive/MyDrive/P2/LLM/fine_tuned_biogpt\')\nprint("Fine-tuned BioGPT model and tokenizer saved successfully!")\n'

# Generate Classification Report

In [20]:
from sklearn.metrics import classification_report
import pandas as pd

# Assume `test_data` is your test dataset after generating predictions
# The test_data dataframe should already have columns: 'Label' (true labels) and 'Predicted' (model predictions)

# True labels and predicted labels
true_labels = test_data['disease_label']
predicted_labels = test_data['predicted_disease']

# Generate classification report
report = classification_report(true_labels, predicted_labels, output_dict=False)
print(report)

# Save classification report to a file
report_dict = classification_report(true_labels, predicted_labels, output_dict=True)
pd.DataFrame(report_dict).transpose().to_csv('/content/drive/MyDrive/P2/T1/Dataset/combined/LLM/ClinicalBERT-c-class.csv', index=True)
print("Classification report saved to '/content/drive/MyDrive/P2/T1/Dataset/combined/LLM/ClinicalBERT-c-class.csv")


                                    precision    recall  f1-score   support

                    adenocarcinoma       0.94      0.91      0.92       183
                          adhesion       0.94      0.93      0.93       176
                     affect labile       0.81      0.86      0.83        83
                         arthritis       0.88      0.93      0.90       139
                            asthma       0.80      0.91      0.85        95
      benign prostatic hypertrophy       0.90      0.95      0.92        83
                  biliary calculus       0.71      0.86      0.78       242
                  carcinoma breast       0.93      0.94      0.94       178
chronic obstructive airway disease       0.77      0.90      0.83       159
                         cirrhosis       0.95      0.88      0.91       194
                           colitis       0.85      0.76      0.80       206
         coronary arteriosclerosis       0.57      0.72      0.64       218
           

In [21]:
"""
output_path = '/content/drive/MyDrive/P2/LLM/ClinicalBERT_Predictions.csv'
test_data.to_csv(output_path, index=False)
print(f"Predictions saved to: {output_path}")
"""

'\noutput_path = \'/content/drive/MyDrive/P2/LLM/ClinicalBERT_Predictions.csv\'\ntest_data.to_csv(output_path, index=False)\nprint(f"Predictions saved to: {output_path}")\n'